In [1]:
import PollingSystem as PS
import MarkovianRP as MRP
import StaticRP as SRP
import numpy as np

from matplotlib import pyplot as plt
from importlib import reload 

In [19]:
reload(PS)
beta = 1/16
els = np.array([0.01, 0.01, 0.01])/beta
#els = np.array([0.3, 0.1, 0.1])/beta
ps = PS.PollingSystem(els, beta)
#S = np.array([[ 0.        , 11.01635149, 14.27059899],
#        [11.01635149,  0.        , 17.132717  ],
#        [14.27059899, 17.132717  ,  0.        ]])

S = np.array([[0,1,1],[1,0,1],[1,1,0]])

P_opt, W_opt = (ps._calc_optiaml_markovian_rp(S))
print(P_opt)
#normalize so no self transitions
print(W_opt)


P = np.array([[0, 0.5, 0.5],
              [0.5, 0, 0.5],
              [0.5, 0.5, 0]])
rp = MRP.MarkovianRP(P)
print(ps.calc_avg_wait(S, rp))
hrs = 1
mins = hrs*60
seconds = mins*60
xt, wt, queues, _, avg_switching, polling_instants, T = ps.simulate(rp, S, seconds)

print(wt[-1][1])

[[0.00000000e+00 9.99419786e-01 5.80213768e-04]
 [5.15567588e-04 0.00000000e+00 9.99484432e-01]
 [9.99298997e-01 7.01002608e-04 0.00000000e+00]]
1.5325117963594967
1.8755369415807563
1.8660120411356085


In [ ]:
print("Empirical backward recurrence times:\n", T)
print("Thr backward recurrence times:\n", ps._T_bar_Markovian(S, P))

In [ ]:
print(ps._stage_durations_mrk(S, P))
#calculate empirical stage durations
total_durations = np.zeros(3)
counts = np.zeros(3)
for i in range(len(polling_instants)-1):
    pli = polling_instants[i]
    nxtpli = polling_instants[i+1]
    q = pli[1]
    counts[q] += 1
    total_durations[q]+= nxtpli[0] - pli[0] - S[q, nxtpli[1]]
    
print(total_durations/counts)

In [ ]:
plgis = np.array(polling_instants)
xt = np.array(xt)
pi = np.array([1/3, 1/3, 1/3])
Lbarmc = 0
sys_lengths = []
sys_lengths_i = []
lbar_i_given_i = np.zeros(3)
for i in range(len(plgis)):
    inst = plgis[i,0]
    z = int(plgis[i,1])
    idx = np.where(xt[:,0]==inst)
#     print(idx)
    idx = idx[0][0]
    sys_lengths.append(sum(xt[idx,2:]))
    sys_lengths_i.append(xt[idx,2:])
    lbar_i_given_i[z] += xt[idx, 2+z]
    Lbarmc += sys_lengths[-1]

print("Lbar MC Emp: %.4f"%(Lbarmc/len(plgis)))
Lmc = ps._LSys_mc_avg(S, pi)
print("lbar mc thr: %.4f"%(Lmc))


sys_lengths_i = np.array(sys_lengths_i)
for i in range(3):
    print("Emp Lbar mc at %d given %d: %.4f"%(i, i, lbar_i_given_i[i]/counts[i] ))
    print("Thr Lbar mc at %d given %d: %.4f"%(i, i, ps._Li_mc_avg_at_i(S, P,i)))

In [ ]:
ps._calc_optiaml_markovian_rp(S)

In [ ]:
x0 = ps._opt_pi_equal_distances()
y = np.tile(x0,3)
def shape(x):
    return np.reshape(x, (3, 3))
shape(y)

In [8]:
P = np.array([[1,4,5],[0.3, 0.5, 0.7],[1, 1, 1]])
(P.T/np.sum(P, axis=1)).T

array([[0.1       , 0.4       , 0.5       ],
       [0.2       , 0.33333333, 0.46666667],
       [0.33333333, 0.33333333, 0.33333333]])

In [7]:
np.sum(P, axis=1).T

array([10. ,  1.5,  3. ])

In [23]:
pi = np.array([1,4,5])
np.tile(pi, (3,1))

array([[1, 4, 5],
       [1, 4, 5],
       [1, 4, 5]])